In [69]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, matthews_corrcoef, confusion_matrix

In [38]:
clinical_attributes = pd.read_csv('step_04/clinical_attributes.csv')
z_score = pd.read_csv('step_04/z_score.csv')
mutation = pd.read_csv('step_04/mutation.csv')
response = pd.read_csv('step_04/response.csv')

In [39]:
class Datasets():
    def __init__(self, seed):
        X_clinical_train_val, self.X_clinical_test, y_train_val, self.y_test = train_test_split(clinical_attributes, response, test_size=0.1, random_state=seed)
        self.X_clinical_train, self.X_clinical_val, self.y_train, self.y_val = train_test_split(X_clinical_train_val, y_train_val, test_size=0.2, random_state=seed)

        X_mutation_train_val, self.X_mutation_test, _, _ = train_test_split(mutation, response, test_size=0.1, random_state=seed)
        self.X_mutation_train, self.X_mutation_val, _, _ = train_test_split(X_mutation_train_val, y_train_val, test_size=0.2, random_state=seed)

        X_z_score_train_val, self.X_z_score_test, _, _ = train_test_split(z_score, response, test_size=0.1, random_state=seed)
        self.X_z_score_train, self.X_z_score_val, _, _ = train_test_split(X_z_score_train_val, y_train_val, test_size=0.2, random_state=seed)

    def get_dataset(self, datasets):
        X_train, X_val, X_test = None, None, None

        if 'clinical' in datasets:
            dataset = {
                'X_train': self.X_clinical_train,
                'X_val': self.X_clinical_val,
                'X_test': self.X_clinical_test
            }

        if 'mutation' in datasets:
            dataset = {
                'X_train': self.X_mutation_train,
                'X_val': self.X_mutation_val,
                'X_test': self.X_mutation_test
            }

        if 'z_score' in datasets:
            dataset = {
                'X_train': self.X_z_score_train,
                'X_val': self.X_z_score_val,
                'X_test': self.X_z_score_test
            }
        
        dataset.update({
            'y_train': np.squeeze(self.y_train),
            'y_val': np.squeeze(self.y_val),
            'y_test': np.squeeze(self.y_test)
        })

        return dataset

In [40]:
data = Datasets(42)

# Halving Search

### Model definition

In [60]:
knn_clinical = KNeighborsClassifier(n_neighbors=3)
svc_mutation = SVC(kernel='linear', C=1, probability=True)
dt_z_score = DecisionTreeClassifier(max_depth=3)

In [61]:
knn_clinical.fit(data.get_dataset(['clinical'])['X_train'], data.get_dataset(['clinical'])['y_train'])
svc_mutation.fit(data.get_dataset(['mutation'])['X_train'], data.get_dataset(['mutation'])['y_train'])
dt_z_score.fit(data.get_dataset(['z_score'])['X_train'], data.get_dataset(['z_score'])['y_train'])

DecisionTreeClassifier(max_depth=3)

In [62]:
voting_clf = VotingClassifier([
    ('knn', knn_clinical),
    ('dt', dt_z_score),
    ('svm', svc_mutation)
], voting='soft')

voting_clf.fit(data.get_dataset(['clinical'])['X_train'], data.get_dataset(['clinical'])['y_train'])

VotingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=3)),
                             ('dt', DecisionTreeClassifier(max_depth=3)),
                             ('svm',
                              SVC(C=1, kernel='linear', probability=True))],
                 voting='soft')

In [71]:
def print_metrics(y_true, predictions, dataset_name):
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, predictions, average='binary')
    accuracy = accuracy_score(y_true, predictions)
    mcc = matthews_corrcoef(y_true, predictions)
    cm = confusion_matrix(y_true, predictions)
    
    print(f"{dataset_name} Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1:.2f}")
    print(f"{dataset_name} Accuracy: {accuracy:.2f}, MCC: {mcc:.2f}")
    print(f"{dataset_name} Confusion Matrix:")
    print(cm)

val_predictions = voting_clf.predict(data.get_dataset(['clinical'])['X_val'])
test_predictions = voting_clf.predict(data.get_dataset(['clinical'])['X_test'])

print_metrics(data.get_dataset(['clinical'])['y_val'], val_predictions, "Validation")

print_metrics(data.get_dataset(['clinical'])['y_test'], test_predictions, "Test")


Validation Precision: 0.71, Recall: 0.63, F1 Score: 0.66
Validation Accuracy: 0.66, MCC: 0.32
Validation Confusion Matrix:
[[68 30]
 [43 72]]
Test Precision: 0.79, Recall: 0.62, F1 Score: 0.69
Test Accuracy: 0.69, MCC: 0.40
Test Confusion Matrix:
[[40 11]
 [26 42]]


# Teapot